In [1]:
!pip install catboost

In [2]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor

In [3]:
def load_merge_data() :
    df_train = pd.read_excel(
    "data/train.xlsx").rename(columns={"dt": "timestamp", "Цена на арматуру": "target"})
    df_test = pd.read_excel(
        "data/test.xlsx").rename(columns={"dt": "timestamp", "Цена на арматуру": "target"})
    df_full = pd.concat([df_train, df_test], ignore_index=True)
    df_chmf = pd.read_csv(
        "data/CHMF Акции.csv").rename(columns={"Date": "timestamp"})
    df_magn = pd.read_csv(
        "data/MAGN Акции.csv").rename(columns={"Дата": "timestamp"})
    df_nlmk = pd.read_csv(
        "data/NLMK Акции.csv").rename(columns={"Date": "timestamp"})
    df_transfer = pd.read_excel("data/Грузоперевозки.xlsx").rename(
        columns={"dt": "timestamp", "Индекс стоимости грузоперевозок": "transfer_cost"})
    df_market = pd.read_excel(
        "data/Данные рынка стройматериалов.xlsx").rename(columns={"dt": "timestamp"})
    df_lme = pd.read_excel(
        "data/Индекс LME.xlsx").rename(columns={"дата": "timestamp"})
    df_macro = pd.read_excel(
        "data/Макропоказатели.xlsx").rename(columns={"dt": "timestamp"})
    df_fuel = pd.read_excel(
        "data/Топливо.xlsx").rename(columns={"dt": "timestamp"})
    df_raw_prices = pd.read_excel(
        "data/Цены на сырье.xlsx").rename(columns={"dt": "timestamp"})

    df_chmf["timestamp"] = pd.to_datetime(df_chmf["timestamp"])
    df_magn["timestamp"] = pd.to_datetime(df_magn["timestamp"])
    df_nlmk["timestamp"] = pd.to_datetime(df_nlmk["timestamp"])

    # Merge the dataframes
    merged_df = pd.merge(df_full, df_chmf, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_magn, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_nlmk, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_transfer, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_market, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_lme, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_macro, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_fuel, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_raw_prices, on='timestamp', how='outer')

    merged_df = merged_df.sort_values("timestamp")

    return merged_df.sort_values("timestamp")

In [4]:
def prepare_data(merged_df):
    merged_df = merged_df.dropna(subset=["target"])
    # Iterate over the columns in the DataFrame
    for column in merged_df.columns:
        if column != "timestamp":
            # Check if the column contains non-numeric values
            if merged_df[column].dtype != float:
                # Extract numeric values using regular expressions
                merged_df[column] = merged_df[column].apply(lambda x: re.findall(
                    r"[-+]?\d*\.\d+|\d+", str(x))[0] if re.findall(r"[-+]?\d*\.\d+|\d+", str(x)) else None)

                # Convert the column to float data type
                merged_df[column] = merged_df[column].astype(float)

    # Convert the "timestamp" column to datetime, if needed
    merged_df["timestamp"] = pd.to_datetime(merged_df["timestamp"])

    nan_counts = merged_df.isna().sum()
    columns_with_high_nan = nan_counts[nan_counts > merged_df.shape[0] * 0.5].index
    merged_df = merged_df.drop(columns=columns_with_high_nan)

    #TODO:fillna

    return merged_df

In [5]:
def make_data_blocks(df, shift, sz): # make a new dataset with data between weeks <t-shift-sz> and <t-shift>, to predict the change of price between week <t> and week <t-shift>
    upgrade_df = df[["timestamp"]].copy()
    rga, rgb = shift, shift+sz

    x = np.array(df["target"]-df["target"].shift(rga)) / 100
    x = x / (np.abs(x)+1) #SoftSign
    upgrade_df["target"] = x

    for col in df.columns:
        if col not in ["timestamp"]:
            for i in range(rga, rgb):
                col1 = col+str(i)
                if i!=rga:
                    upgrade_df[col1] = df[col].shift(i)-df[col].shift(rga)
                else:
                    upgrade_df[col1] = df[col].shift(i)

    upgrade_df = upgrade_df[10:]
    return upgrade_df

In [6]:
test_size = 28

def train_test_split(df): #cut test data from the end of the dataset
    return (df[:-test_size].copy().reset_index(drop=True), df[-test_size:].copy().reset_index(drop=True))

In [7]:
class model: #contains the model, and data, prepared for it (different models require different states of data)
    def __init__(self, shift, sz, name):
        df = prepare_data(load_merge_data())
        upgrade_df = make_data_blocks(df, shift, sz)
        upgrade_train, upgrade_test = train_test_split(upgrade_df)
        self.test_y = upgrade_test['target']
        self.test_x = upgrade_test.drop(columns=['target', "timestamp"])
        self.train_y = upgrade_train['target']
        self.train_x = upgrade_train.drop(columns=['target', "timestamp"])
        self.name = name

    def save(self):
        self.model.save_model(self.name)

    def load(self):
        self.model = CatBoostRegressor(verbose=0)
        self.model.load_model(self.name)

    def fit(self):
        self.model = CatBoostRegressor(verbose=0)#, eval_metric = "R2")
        self.model.fit(self.train_x , self.train_y)

    def predict(self):
        return self.model.predict(self.test_x)

    def true(self):
        return self.test_y

In [8]:
def find_negative_index(lst):
    for i, num in enumerate(lst):
        if num < 0:
            return i
    return len(lst)

def make_orders_on_time_segment(models):
    result = []
    step = 0
    models_predictions = [model.predict() for model in models]
    for day_ind in range(test_size):
        if step == 0:
            segment = [models_predictions[i][day_ind] for i in range(len(models))]
            step = find_negative_index(segment)+1
            result.append(min(step, test_size-day_ind))
        else:
            result.append(0)
        step-=1
    return result

In [9]:
def decision_prices(test):
    test = test.set_index('dt')
    tender_price = test['Цена на арматуру']
    decision = test['Объем']
    start_date = test.index.min()
    end_date = test.index.max()

    _results = []
    _active_weeks = 0
    for report_date in pd.date_range(start_date, end_date, freq='W-MON'):
        if _active_weeks == 0:  # Пришла пора нового тендера
            _fixed_price = tender_price.loc[report_date]
            _active_weeks = int(decision.loc[report_date])
        _results.append(_fixed_price)
        _active_weeks += -1
    cost = sum(_results)
    return cost # Возвращаем затраты на периоде

In [10]:
block_sz = 3 #hyperparam 3 is outputted in the train notebook (we didn't save it in the model file yet) (and it is probably wrong)

In [11]:
models = []
for i in range(1, 10):
    models.append(model(i, block_sz, "models/cb_model_" + str(i)))
    models[-1].load()

C:\Users\Vitya\AppData\Local\Temp\ipykernel_10740\4147000836.py:27: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_magn["timestamp"] = pd.to_datetime(df_magn["timestamp"])
C:\Users\Vitya\AppData\Local\Temp\ipykernel_10740\4161664041.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[column] = merged_df[column].apply(lambda x: re.findall(
C:\Users\Vitya\AppData\Local\Temp\ipykernel_10740\4161664041.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [12]:
res = make_orders_on_time_segment(models)
print(*res)
test = pd.read_excel('./data/test.xlsx')
print(test)
test['Объем'] = res
test.to_excel('submission.xlsx', index=False)

decision_prices(test)

1 1 1 1 2 0 3 0 0 1 1 1 1 1 2 0 5 0 0 0 0 5 0 0 0 0 2 0
           dt  Цена на арматуру
0  2022-09-05             46000
1  2022-09-12             46000
2  2022-09-19             46000
3  2022-09-26             45500
4  2022-10-03             45500
5  2022-10-10             45000
6  2022-10-17             44000
7  2022-10-24             43000
8  2022-10-31             42000
9  2022-11-07             41500
10 2022-11-14             41000
11 2022-11-21             40000
12 2022-11-28             40000
13 2022-12-05             40500
14 2022-12-12             41700
15 2022-12-19             41700
16 2022-12-26             42500
17 2023-01-02             42700
18 2023-01-09             42700
19 2023-01-16             42700
20 2023-01-23             42800
21 2023-01-30             42800
22 2023-02-06             44500
23 2023-02-13             44500
24 2023-02-20             46000
25 2023-02-27             49200
26 2023-03-06             53700
27 2023-03-13             55500


1226800